In [ ]:
# Import libraries
import pickle
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import WordPunctTokenizer
from collections import Counter
from string import punctuation, ascii_lowercase
import re
import joblib
from tqdm import tqdm
from gensim.models import Word2Vec
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout,SpatialDropout1D, Bidirectional
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from keras.metrics import accuracy,categorical_accuracy
from keras.initializers import Constant

In [ ]:
# prodata4 is preprocessed Data
# only run when want to train on data4


# df = pd.read_pickle("PreProcessed_data/prodata4.pkl")
# df=df.rename(columns={"label":"sentiment"})
# print(df)

In [ ]:
# prodata3 is preprocessed Data
# only run when want to train on data3


# df=pd.read_pickle("PreProcessed_data/prodata3.pkl")
# print(df.sentiment.unique())
# print(df.sentiment.value_counts())

In [ ]:
# To remove label surprise from prodata3


# df=df[df.sentiment!='surprise']
# print(df.sentiment.unique())
# print(df.sentiment.value_counts())

In [ ]:
# # tweet_data2_all.pkl is preprocessed Data2


# df=pd.read_pickle('PreProcessed_data/tweet_data2_all.pkl')
# df['content'] = [' '.join(map(str,l)) for l in df['content']]

# # To remove following label from data2

# df=df[df.sentiment!='neutral']
# df=df[df.sentiment!='love']
# df=df[df.sentiment!='surprise']
# print(df)

In [ ]:
# Reading combined data of all above dataset after preprocessing
# only run when want to train on combined data

#df=pd.read_pickle('PreProcessed_data/combineData.pkl')
#print(df)

In [ ]:
# label value counts

print(df.sentiment.value_counts())

joy        6200
fear       6188
sadness    5695
anger      5206
Name: sentiment, dtype: int64


In [ ]:
# local Preprocessing functions whih was not done previously

replace_puncts = {'`': "'", '′': "'", '“':'"', '”': '"', '‘': "'"}

strip_chars = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '[', ']', '>', '=', '+', '\\', '•',  '~', '@', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

puncts = ['!', '?', '$', '&', '/', '%', '#', '*','£']

def clean_str(x):
    x = str(x)
    
    x = x.lower()
    
    x = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", x)
    
    for k, v in replace_puncts.items():
        x = x.replace(k, f' {v} ')
        
    for punct in strip_chars:
        x = x.replace(punct, ' ') 
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
        
    x = x.replace(" '", " ")
    x = x.replace("' ", " ")
        
    return x

In [ ]:
# applying preprocessing

df['content'] = df['content'].apply(clean_str)
print(df['content'])

In [ ]:
print(df.sentiment.value_counts())

joy        6200
fear       6188
sadness    5695
anger      5206
Name: sentiment, dtype: int64


In [ ]:
# data analysis for further use in model creation

df['l'] = df['content'].apply(lambda x: len(str(x).split(' ')))
print("mean length of sentence: " + str(df.l.mean()))
print("max length of sentence: " + str(df.l.max()))
print("std dev length of sentence: " + str(df.l.std()))

mean length of sentence: 13.326420198376915
max length of sentence: 136
std dev length of sentence: 9.966683924947404


In [ ]:
# sequence length of tweet for training and prediction

sequence_length = 136

In [ ]:
max_features = 20000 # this is the number of words we care about

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(df['content'].values)

# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(df['content'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)

y = pd.get_dummies(df['sentiment']).values

# lets keep a couple of thousand samples back as a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6,random_state=41)

print("test set size " + str(len(X_test)))

test set size 13974


In [ ]:
# word embedding from Glove
# download from http://nlp.stanford.edu/data/glove.twitter.27B.zip

embeddings_index = {}
f = open('../glove.twitter.27B/glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193515 word vectors.


In [ ]:
# unique tokens in combined data

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19273 unique tokens.


In [ ]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 100

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

19274


In [ ]:
# model creation

model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=sequence_length,
                    trainable=True))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(units=4, activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 136, 100)          1927400   
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 136, 100)          0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 136, 256)          234496    
_________________________________________________________________
dropout_11 (Dropout)         (None, 136, 256)          0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 256)               394240    
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                

In [ ]:
# Train model

batch_size = 128

history = model.fit(X_train, y_train, epochs=6, batch_size=batch_size, verbose=1, validation_split=0.15)

/home/administrator/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 19795 samples, validate on 3494 samples
Epoch 1/6
19795/19795 [==============================] - 135s 7ms/step - loss: 1.1468 - accuracy: 0.4918 - val_loss: 1.3920 - val_accuracy: 0.4033
Epoch 2/6
19795/19795 [==============================] - 133s 7ms/step - loss: 0.9073 - accuracy: 0.6358 - val_loss: 1.3054 - val_accuracy: 0.4313
Epoch 3/6
19795/19795 [==============================] - 133s 7ms/step - loss: 0.7435 - accuracy: 0.7119 - val_loss: 1.1740 - val_accuracy: 0.5169
Epoch 4/6
19795/19795 [==============================] - 134s 7ms/step - loss: 0.6386 - accuracy: 0.7605 - val_loss: 0.9709 - val_accuracy: 0.6251
Epoch 5/6
19795/19795 [==============================] - 136s 7ms/step - loss: 0.5550 - accuracy: 0.7892 - val_loss: 1.1519 - val_accuracy: 0.5985
Epoch 6/6
19795/19795 [==============================] - 135s 7ms/step - loss: 0.4822 - accuracy: 0.8190 - val_loss: 1.1311 - val_accuracy: 0.6216


In [ ]:
print(X_test.shape,y_test.shape)

(2593, 136) (2593, 4)


In [ ]:
# load model from local disk if available else save from below code

model = joblib.load('model_combinedata_83.pkl')

c:\users\harsh\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
# loss and accuracy score

model.evaluate(X_test, y_test, verbose=0)

[0.4626077514435577, 0.8391298055648804]

In [ ]:
# Save the model as a pickle file on disk 
joblib.dump(model, 'model_combinedata_83.pkl')

['model_combinedata_83.pkl']